In [137]:
from collections import defaultdict
import re
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import accuracy_score
import random
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

creer_fichier_embeddings = False

#1) Extraction des données d'entraînement

#Eléments à définir
data_path = "FSE-1.1-191210/FSE-1.1.data.xml"
gold_path = "FSE-1.1-191210/FSE-1.1.gold.key.txt"
context_size = 4
embeddings_path = "embeddings.txt" #Saisir le path du fichier existant ou le nom de celui qui sera crée dans le cas échéant

#récupération des données XML
tree = ET.parse(data_path)
data_file = tree.getroot()[0]

#récupération des données .txt
gold_file = open(gold_path, "r",encoding="utf-8")

def extract_examples_and_senses(data_file, gold_file):
    """Extract the data from the files.

    Args:
        data_file (Element): Sentences
        gold_file (TextIOWrapper): Golds keys

    Returns:
        dictionary: associates the list of context vectors corresponding to the instance
        dictionary : associates to the word each senses
    """
    
    w2examples={}
    w2senses = defaultdict(set)
    
    for (sentence,gold_line) in zip(data_file,gold_file.readlines()) :
        
        #pour chaque phrase, on initialise deux listes qui permettront de respecter les tailles des contextes (+10,-10)
        context_before = []
        context_after = []
        context = []
        
        #on boucle sur les mots de la phrase pour construire les listes
        #on cherche l'instance et on repart en arrière pour constuire le contexte avant
        i_instance = 0
        while sentence[i_instance].tag != "instance" : 
            i_instance+=1
        
        instance = sentence[i_instance].attrib["lemma"].lower()
        
        if instance not in w2examples : 
            w2examples[instance] = []
        
        #on vérifie la longueur des phrases pour ne pas soulever d'erreur
        
        #context_before 
        
        #si le contexte avant l'instance est supérieur ou égale à la taille du contexte choisie
        #on ajoute à la liste chaque mot aux index from i-instance-1 to i_instance-5
        if (len(sentence[:i_instance])>=context_size) :
                for i in range(1,context_size+1) :
                    context_before.append(sentence[i_instance-i].text.lower())
        
        #sinon, on ajoute à la liste tous les mots et on ajoutera des balises pour compléter
        else :
            for i in range(1,len(sentence[:i_instance])+1) :
                context_before.append(sentence[i_instance-i].text.lower())

        #context_after
        
        #si le contexte après l'instance est supérieur ou égale à la taille du contexte choisie
        #on ajoute à la liste chaque mot aux index from i-instance+1 to i_instance+11
        if(len(sentence[i_instance+1:])>= context_size) :
            for i in range(i_instance+1,i_instance+(context_size+1)):
                context_after.append(sentence[i].text.lower())
        
        #sinon, on ajoute à la liste tous les mots et on ajoutera des balises pour compléter
        else :
            for i in range(i_instance+1,len(sentence)):
                context_after.append(sentence[i].text.lower())
        
        #une fois les listes constituées, on ajoute les balises de début et de fin de phrase si nécessaire
        for i in range(context_size-len(context_before)) :
            context_before.append("<d>")
            
        for i in range(context_size-len(context_after)) :
            context_after.append("<f>")
            
        #le vecteur sera une concaténation des contextes d'avant et d'après
        context = context_before
        context.append(instance)
        context.extend(context_after)
            
        #on récupère ensuite le nombre associé au sens pour constuire l'exemple + ajouter au dictionnaire w2sense
        gold = int((re.findall("ws_[0-9]",gold_line)[0]).replace("ws_",""))
        
        w2senses[instance].add(gold)
        w2examples[instance].append((context,gold))
        
    return w2examples,w2senses

w2examples,w2senses = extract_examples_and_senses(data_file,gold_file)
instances = list(w2examples.keys())
print("nombre d'instances présentes dans le corpus: ",len(instances),"\n")
print("instance et taille des données d'entraînement",[(word,len(w2examples[word])) for word in w2examples],"\n")

#2) Facultatif : script pour créer le fichier texte ne comportant que les embeddings qui nous intéressent - plus rapide

if creer_fichier_embeddings :
    #Element à définir
    fasstex_path = "../fasstex" 

    #création du vocabulaire du corpus entier
    i2w = set()
    for instance,examples in w2examples.items():
        for context,gold in examples :
            i2w.update(context)
    i2w = list(i2w)

    f = open(fasstex_path, "r", encoding="UTF-8")
    f.readline() #permet de ne pas prendre en compte la première ligne du fichier qui résumé ce que contient le fichier
    lines = f.readlines()

    with open(embeddings_path,"w",encoding="UTF-8") as f : 
            f.writelines([line for line in lines if line.split(" ")[0] in i2w])

#3) Construction du dictionnaire qui va nous permettre de faire le look up (matrice d'embeddings)

def extract_embeddings(path_embeddings) :
    """Récupère les embeddings dans le fichier générée.

    Args:
        path_embeddings (string)

    Returns:
        dictionnary: Associe à chaque mot son embedding
    """
    f = open(path_embeddings , "r", encoding="UTF-8")

    #On récupère dans le fichier crée les embeddings pour créer un dictionnaire
    w2emb = {}
    for line in f.readlines():
        splitted_line = line.split(" ")
        word = splitted_line[0]
        embedding = list(map(float,splitted_line[1:]))
        w2emb[word] = embedding
    return w2emb

w2emb = extract_embeddings(embeddings_path)
#4) Sélection des instances pour la classification et opération de look up pour chacune d'elle

#Elément à définir #EST-CE QUE LA ON MET LA FONCTION MAKE_BATCHES ?
instances_to_test = instances[:3]


def look_up(context, w2emb) :
    """Remplace dans le vecteur de contexte les mots par leur embedding.

    Args:
        context (list): liste de taille (size_window*2)+1
        w2emb (dictionnary): Associe à chaque mot son embedding

    Returns:
        list : liste de taille size_embedding : BOW
    """
    emb_size = len(list(w2emb.values())[0]) #on récupère la taille d'un embedding : 300
    context_emb = np.zeros(emb_size)
    for word in context :
        if word in w2emb :
            context_emb = np.add(context_emb, np.array(w2emb[word]))             
    return context_emb

#5) Sélection de données représentatives du corpus d'entraînement en fonction du nombre de données considérées
size_data = [1,0.8,0.6,0.4,0.2]
#Remarque : "affecter" compte 7 senses différents pour 25 examples et "demeurer",6 senses pour 21 examples.
#Conséquence : on ne peut pas considérer moins de 25% des données annotées pour "affecter" et 30% pour "demeurer"

def select_examples(examples,senses,size):
    """Choisit des examples d'entraînement représentatifs du corpus.

    Args:
        examples (list)
        senses (int): nombre de senses associés à l'instance
        size (float): quantité des données d'entraînement considérés

    Returns:
        list: examples qui contiennent au moins un example de chaque sense
    """
    selected_examples = []
    
    #Pour chaque sens, on ajoute un example associé à ce sens ,au hasard
    for sense in senses :
        selected_examples.append(random.choice(list(filter((lambda example:example[1]==sense),examples))))
    
    #On calcule ensuite le nombre d'examples qu'il reste à ajouter pour atteindre la quantité de données souhaitée
    size_to_add = round(size*(len(examples)))-len(selected_examples)
    
    #On ajoute ce nombre de données (non-présentes déjà dans la liste) selectionnées au hasard
    selected_examples.extend(random.choices(list(filter((lambda example : example not in selected_examples),examples)),k=size_to_add))
    
    return selected_examples

nombre d'instances présentes dans le corpus:  37 

instance et taille des données d'entraînement [('aboutir', 25), ('investir', 25), ('traduire', 25), ('témoigner', 25), ('juger', 12), ('justifier', 25), ('viser', 25), ('prononcer', 25), ('accomplir', 25), ('convenir', 25), ('acquérir', 25), ('achever', 25), ('observer', 25), ('adapter', 25), ('admettre', 23), ('entraîner', 25), ('payer', 25), ('respecter', 24), ('affecter', 25), ('demeurer', 21), ('aggraver', 25), ('agir', 25), ('ajouter', 25), ('alimenter', 25), ('coûter', 22), ('relancer', 25), ('préférer', 25), ('appliquer', 25), ('apporter', 25), ('fonder', 25), ('appuyer', 25), ('changer', 22), ('chuter', 11), ('soutenir', 25), ('concevoir', 25), ('interroger', 25), ('confirmer', 25)] 



In [140]:
import numpy as np
import torch
import pandas as pd
import random

class K_Means():
    ''' 
    classifieur K-means pour un mot particulier
    '''

    def __init__(self, annotated_examples, not_annotated_examples):
        '''
        Instancie les différentes variables utiles pour l'algorithme du K-means

        examples : liste d'examples dont le mot à désambiguiser est le même pour 
                   chaque example
        example : couple d'un mot avec son contexte de fenêtre 4 (sous forme 
                  d'embedding) et du numéro de sens attendu du mot à désambiguiser 
                  (gold class sous forme d'integer)
                    si example = ([1.9, 2.3, 0.6], 1),
                    - le contexte avec le mot à désambiguiser et son lemme est 
                      l'embedding [1.9, 2.3, 0.6]
                    - le numéro de sens est 1
        '''

        # transforme l'ensemble des examples en une liste pour pouvoir garder le 
        # même indice pour chaque example par la suite
        self.annotated_examples = annotated_examples

        # on a le gold, mais on ne l'utilisera que pour calculer l'accuracy
        # pour le training, nous ferons comme si nous n'avions pas la gold class
        self.not_annotated_examples = not_annotated_examples

        # transforme les embeddings en tensors
        # ce sont les exemples qui devront être classifiés
        self.tensors_examples = [example[0] for example in self.not_annotated_examples]

        # détermine le nombre de sens possibles k (donc le nombre de clusters) 
        # à l'aide des données annotées, qui représentent tous les sens possibles
        self.k = len(set([example[1] for example in self.annotated_examples]))

        
        # initialisation des centroïdes : pour chaque sens, le centroïde 
        # correspond à la moyenne des embeddings des exemples annotés
        # Ainsi, chaque centroïde représente un sens
        self.tensors_centroids, self.cluster2sense = self.make_centroids()

        # initialisation de clusters : tous les examples sont associés au cluster 0
        self.clusters = np.zeros(len(not_annotated_examples))



    def make_centroids(self):
        cluster2sense = []
        tensors_centroids = []
        senses = set([example[1] for example in self.annotated_examples])
        for sense in senses: 
            # on récupère les examples du sens
            examples_sense = [example[0] for example in self.annotated_examples if example[1] == sense]
            # on calcule le centroid du sens
            centroid = torch.mean(torch.stack(examples_sense), dim=0)
            # on ajoute le centroid à la liste des centroids
            tensors_centroids.append(centroid)
            # on ajoute le sens à la liste des sens
            #L'index du sens correspond au numéro du cluster
            cluster2sense.append(sense)

        return tensors_centroids, cluster2sense


    def learn_clusters(self):
        '''
        Algorithme de K-Means
        Retourne les coordonnées de chaque centroide ainsi que le cluster auquel 
        appartient chaque example
        '''

        # différence initialisée à Vrai
        diff = True
        
        # tant qu'il y a une différence entre l'ancienne liste et la nouvelle 
        # liste de centroides
        while diff:

            # CALCUL DES DISTANCES ENTRE CHAQUE EXAMPLE ET CHAQUE CENTROIDE

            # pour chaque couple (indice, coordonnées) dans les examples
            for i, tensor_example in enumerate(self.tensors_examples):
                # initialisation de la distance minimum à l'infini
                min_dist = float('inf')
                # pour chaque couple (indice, coordonnées) dans les centroides
                for j, tensor_centroid in enumerate(self.tensors_centroids):
                    # calcul de la distance entre cet example et ce centroide
                    d = (tensor_centroid - tensor_example).pow(2).sum(axis=0).sqrt()
                    # si une distance plus faible est trouvée
                    if min_dist > d:
                        # la distance ainsi que le centroide sont stockés
                        min_dist = d
                        self.clusters[i] = j
            
            # CALCUL DES NOUVEAUX CENTROIDES

            # calcul des nouveaux centroides en utilisant le point au milieu de tous les
            # autres points du même cluster
            new_centroids = pd.DataFrame(self.tensors_examples).groupby(by = self.clusters).mean()
            # transforme ces nouveaux centroides en tensors
            tensors_new_centroids = []
            for i in range(len(new_centroids.index)):
                colums = []
                for j in range(len(new_centroids.columns)):
                    colums.append(int(new_centroids.iat[i,j]))
                tensors_new_centroids.append(torch.tensor(colums))

            # MISE A JOUR DES CENTROIDES
            '''J'ai essayé de régler le problème de range mais possible que yen ait tjrs '''
            count_diff = 0
            for i in range(len(tensors_new_centroids)):
                if not torch.equal(self.tensors_centroids[i], tensors_new_centroids[i]):
                    count_diff += 1
                    self.tensors_centroids[i] = tensors_new_centroids[i]
            if count_diff == 0:
                diff = False
            
            ''' Ancien bout de code de Mathilde
            count_diff = 0
            # pour chaque centroide
            for i in range(len(self.tensors_centroids)):
                # si l'ancien centroide et le nouveau ne sont pas les mêmes
                if not(torch.equal(self.tensors_centroids[i], tensors_new_centroids[i])):
                    count_diff += 1
                    # met à jour le centroide
                    self.tensors_centroids = tensors_new_centroids
            # s'il n'y a eu aucune différence entre les anciens et les nouveaux centroides, 
            # la boucle while se termine
            if count_diff == 0:
                diff = False
            '''
            

    def accuracy(self): 
        correct = 0
        for i in range(len(self.not_annotated_examples)):
            sens_attendu = self.not_annotated_examples[i][1]
            sens_assigne = self.cluster2sense[int(self.clusters[i])]
            #print(sens_assigne, sens_attendu)
            if sens_attendu == sens_assigne:
                correct += 1
        return correct, len(self.not_annotated_examples)
    

In [141]:


for size in size_data :

    df_size_data = []

    list_num_correct = []
    list_num_examples = []
    
    print(f'{size*100}% de données annotées')

    for instance in instances_to_test :
            
        print("--> Instance :",instance)
        
        annotated_examples = select_examples(w2examples[instance],w2senses[instance],size)

        not_annotated_examples = [ex for ex in select_examples(w2examples[instance],w2senses[instance],100) if ex not in annotated_examples]

        
        
        X_annotated = [torch.from_numpy(look_up(context,w2emb)) for context,gold in annotated_examples]
        y_annotated = [gold for context,gold in annotated_examples]

        X = [torch.from_numpy(look_up(context,w2emb)) for context,gold in not_annotated_examples]
        y_gold = [gold for context,gold in not_annotated_examples]
        
        y = y_annotated + y_gold
        #print(y)
        most_frequent_sense = max(y,key=y.count)
        occurrence_of_most_frequent_sense = y.count(most_frequent_sense)/len(y)
        print(f"Le sens le plus fréquent pour '{instance}' est le sens {most_frequent_sense} avec une proportion de {round(occurrence_of_most_frequent_sense*100,2)} %")
        
        annotated_examples = [(X_annotated[i],y_annotated[i]) for i in range(len(X_annotated))]
        not_annotated_examples = [(X[i],y_gold[i]) for i in range(len(X))]
        k_Means = K_Means(annotated_examples, not_annotated_examples)

        k_Means.learn_clusters()
        num_correct, num_examples = k_Means.accuracy()
        print(f"Avec le kmeans et {size*100}% des données annotées pour '{instance}', l'accuracy est de {round(num_correct/num_examples*100,2)} %")
        print()
        
        list_num_correct.append(num_correct)
        list_num_examples.append(num_examples)
        
    micro_average = sum(list_num_correct)/sum(list_num_examples)
    print(f"Micro-average : {round(micro_average*100,2)} %")
    print("\n--------------------------------------------------\n")

        


    


100% de données annotées
--> Instance : aboutir
Le sens le plus fréquent pour 'aboutir' est le sens 3 avec une proportion de 99.69 %
Avec le kmeans et 100% des données annotées pour 'aboutir', l'accuracy est de 0.0 %

--> Instance : investir
Le sens le plus fréquent pour 'investir' est le sens 3 avec une proportion de 61.9 %
Avec le kmeans et 100% des données annotées pour 'investir', l'accuracy est de 12.45 %

--> Instance : traduire
Le sens le plus fréquent pour 'traduire' est le sens 1 avec une proportion de 80.55 %
Avec le kmeans et 100% des données annotées pour 'traduire', l'accuracy est de 20.85 %

Micro-average : 9.96 %

--------------------------------------------------

80.0% de données annotées
--> Instance : aboutir
Le sens le plus fréquent pour 'aboutir' est le sens 3 avec une proportion de 99.73 %
Avec le kmeans et 80.0% des données annotées pour 'aboutir', l'accuracy est de 0.0 %

--> Instance : investir
Le sens le plus fréquent pour 'investir' est le sens 4 avec une pro

KeyboardInterrupt: 